# Pose-Based Form Checker (RTSP Stream) – **IPython Real-Time Display**

*Reference animations from pre-recorded videos.*  
*Live RTSP video (laptop → OBS/FFmpeg) → pose detection → **blue = reference, green/red = user**.*  
*Real-time feedback **inside the notebook** using `IPython.display` + JPEG streaming (no Flask, no external browser).*

---

## Setup

1. **Start RTSP on laptop** (low-latency):
   ```bash
   ffmpeg -f dshow -i video="YOUR_WEBCAM_NAME" -c:v libx264 -preset ultrafast -tune zerolatency -profile:v baseline -pix_fmt yuv420p -maxrate 1000k -bufsize 2000k -g 30 -f rtsp rtsp://0.0.0.0:8554/webcam.sdp
   ```

2. **Update `RTSP_URL`** if your laptop IP changes.

3. Run the notebook – the live stick-figure overlay appears **right below**.

---

**Folder structure**
```
project/
├── this_notebook.ipynb
└── exercises/
    ├── squat.mp4
    ├── push-up.mp4
    └── ...
```

In [ ]:
import cv2
import mediapipe as mp
import numpy as np
import pickle
import time
import os
import glob
from IPython import display as ipydisplay

mp_pose = mp.solutions.pose

# ----------------------------------------------------
# RTSP configuration
# ----------------------------------------------------
RTSP_URL = "rtsp://10.227.207.170:8554/webcam.sdp"   # <-- change if needed

# Improved FFmpeg options to handle stream errors gracefully
os.environ["OPENCV_FFMPEG_CAPTURE_OPTIONS"] = (
    "rtsp_transport;tcp|"           # force TCP
    "fflags;nobuffer+discardcorrupt|"  # discard corrupted frames
    "flags;low_delay|"              # low-delay decoding
    "max_delay;0|"                  # zero delay
    "err_detect;ignore_err"         # ignore decode errors
)


## Helper: angle calculation

In [ ]:
def calculate_angle(a, b, c):
    a = np.array(a)
    b = np.array(b)
    c = np.array(c)
    ab = a - b
    bc = c - b
    cosine_angle = np.dot(ab, bc) / (np.linalg.norm(ab) * np.linalg.norm(bc) + 1e-6)
    angle = np.arccos(np.clip(cosine_angle, -1.0, 1.0))
    return np.degrees(angle)

## Extract reference landmarks from a video

In [ ]:
def extract_reference_from_video(video_path: str, exercise_name: str, target_fps: float = 10.0):
    if not os.path.exists(video_path):
        print(f"Video not found: {video_path}")
        return

    pose = mp_pose.Pose(static_image_mode=False,
                        model_complexity=1,
                        enable_segmentation=False,
                        min_detection_confidence=0.5,
                        min_tracking_confidence=0.5)

    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print(f"Cannot open video: {video_path}")
        return

    video_fps = cap.get(cv2.CAP_PROP_FPS)
    frame_interval = max(1, int(video_fps / target_fps))
    references = []
    frame_count = 0

    print(f"Extracting reference from {video_path} @ ~{target_fps} FPS...")

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        if frame_count % frame_interval == 0:
            rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            results = pose.process(rgb)
            if results.pose_landmarks:
                lm_list = [(lm.x, lm.y, lm.z, lm.visibility) for lm in results.pose_landmarks.landmark]
                references.append(lm_list)

        frame_count += 1

    cap.release()
    pose.close()

    pkl_path = f"{exercise_name}.pkl"
    with open(pkl_path, 'wb') as f:
        pickle.dump(references, f)

    print(f"Saved {len(references)} reference frames → {pkl_path}")

## Auto-extract **all** videos in `exercises/`

In [ ]:
def extract_all_references_from_folder(folder='exercises', target_fps=10.0):
    if not os.path.exists(folder):
        print(f"Folder '{folder}' not found!")
        return

    video_extensions = ['*.mp4', '*.avi', '*.mov', '*.mkv', '*.wmv']
    video_files = []
    for ext in video_extensions:
        video_files.extend(glob.glob(os.path.join(folder, ext)))

    if not video_files:
        print("No videos found in exercises/ folder.")
        return

    print(f"Found {len(video_files)} video(s). Extracting references...\n")
    for video_path in video_files:
        name = os.path.splitext(os.path.basename(video_path))[0]
        extract_reference_from_video(video_path, name, target_fps)
    print("\nAll references extracted!")

## Run **once** – extract every reference

In [ ]:
extract_all_references_from_folder()

## Live pose comparison **with IPython real-time display**

In [ ]:
def perform_exercise(exercise_name: str):
    pkl_path = f"{exercise_name}.pkl"
    if not os.path.exists(pkl_path):
        print(f"Reference not found: {pkl_path}. Run extraction first!")
        return

    with open(pkl_path, 'rb') as f:
        reference_sequence = pickle.load(f)
    if not reference_sequence:
        print("Empty reference sequence.")
        return

    pose = mp_pose.Pose()

    # ---------- RTSP capture (fallback to GStreamer) ----------
    print("Connecting to RTSP stream...")
    cap = cv2.VideoCapture(RTSP_URL, cv2.CAP_FFMPEG)
    cap.set(cv2.CAP_PROP_BUFFERSIZE, 1)

    if not cap.isOpened():
        print("FFmpeg failed → trying GStreamer...")
        gst = (
            f"rtspsrc location={RTSP_URL} latency=0 protocols=tcp ! "
            "rtph264depay ! h264parse ! avdec_h264 ! "
            "videoconvert ! video/x-raw,format=BGR ! appsink drop=1"
        )
        cap = cv2.VideoCapture(gst, cv2.CAP_GSTREAMER)

    if not cap.isOpened():
        print(f"ERROR: Cannot open {RTSP_URL}")
        return

    print("Connected! Starting live display...")
    cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
    cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)

    # ---------- IPython display placeholder ----------
    display_handle = ipydisplay.display(ipydisplay.Image(data=b''), display_id=True)

    # ---------- Animation control ----------
    ref_idx = 0
    anim_fps = 10
    frame_delay = 1.0 / anim_fps
    last_ref_time = time.time()
    angle_thr = 20

    pose_connections = mp_pose.POSE_CONNECTIONS

    frame_cnt = 0
    err_cnt = 0
    max_err = 10

    try:
        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                err_cnt += 1
                if err_cnt >= max_err:
                    print("Too many dropped frames → stopping.")
                    break
                time.sleep(0.1)
                continue
            err_cnt = 0
            frame_cnt += 1

            # Process every 2nd frame to reduce CPU load
            if frame_cnt % 2 != 0:
                continue

            h, w, _ = frame.shape
            rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            results = pose.process(rgb)

            # Blank canvas
            canvas = np.zeros((h, w, 3), dtype=np.uint8)

            # ----- Draw REFERENCE (BLUE) -----
            ref_lm = reference_sequence[ref_idx]
            for conn in pose_connections:
                s, e = ref_lm[conn[0]], ref_lm[conn[1]]
                if s[3] > 0.1 and e[3] > 0.1:
                    cv2.line(canvas,
                             (int(s[0] * w), int(s[1] * h)),
                             (int(e[0] * w), int(e[1] * h)),
                             (255, 0, 0), 2)   # BGR blue

            # ----- Draw USER (GREEN / RED) -----
            if results.pose_landmarks:
                user_lm = [(lm.x, lm.y, lm.z, lm.visibility) for lm in results.pose_landmarks.landmark]
                for conn in pose_connections:
                    i1, i2 = conn
                    r1, r2 = ref_lm[i1], ref_lm[i2]
                    u1, u2 = user_lm[i1], user_lm[i2]

                    if all(x[3] > 0.1 for x in [r1, r2, u1, u2]):
                        parent_idx = None
                        for c in pose_connections:
                            if c[1] == i2 and c[0] != i1:
                                parent_idx = c[0]; break
                            if c[0] == i2 and c[1] != i1:
                                parent_idx = c[1]; break
                        if parent_idx is not None and ref_lm[parent_idx][3] > 0.1 and user_lm[parent_idx][3] > 0.1:
                            rp, up = ref_lm[parent_idx], user_lm[parent_idx]
                            ref_ang = calculate_angle((r1[0], r1[1]), (r2[0], r2[1]), (rp[0], rp[1]))
                            usr_ang = calculate_angle((u1[0], u1[1]), (u2[0], u2[1]), (up[0], up[1]))
                            color = (0, 255, 0) if abs(ref_ang - usr_ang) < angle_thr else (0, 0, 255)
                            cv2.line(canvas,
                                     (int(u1[0] * w), int(u1[1] * h)),
                                     (int(u2[0] * w), int(u2[1] * h)),
                                     color, 2)

            # ----- Encode to JPEG and update display -----
            _, buffer = cv2.imencode('.jpeg', canvas)
            jpeg_data = buffer.tobytes()
            display_handle.update(ipydisplay.Image(data=jpeg_data))

            # ----- Advance reference animation -----
            if time.time() - last_ref_time >= frame_delay:
                last_ref_time = time.time()
                ref_idx = (ref_idx + 1) % len(reference_sequence)

            time.sleep(0.01)  # Prevent 100% CPU

    except KeyboardInterrupt:
        print("\nStream stopped by user.")
    finally:
        cap.release()
        ipydisplay.clear_output()
        print("Cleanup complete.")

## Start a workout (filename **without** extension)

In [ ]:
# Examples:
# perform_exercise('squat')
# perform_exercise('push-up')
# perform_exercise('romanian_deadlift')

perform_exercise('romanian_deadlift')   # change to your exercise